In [18]:
import pyaudio
import wave

# Set the recording parameters
sample_rate = 16000
num_channels = 1
chunk_size = 1024
record_seconds = 30
output_file = "/recording2.wav"

# Initialize PyAudio
audio = pyaudio.PyAudio()

# Open the audio stream
stream = audio.open(format=pyaudio.paInt16,
                     channels=num_channels,
                     rate=sample_rate,
                     input=True,
                     frames_per_buffer=chunk_size)

# Start recording
frames = []
for i in range(0, int(sample_rate / chunk_size * record_seconds)):
    data = stream.read(chunk_size)
    frames.append(data)

# Stop recording
stream.stop_stream()
stream.close()
audio.terminate()

# Save the recording to a WAV file
wave_file = wave.open(output_file, 'wb')
wave_file.setnchannels(num_channels)
wave_file.setsampwidth(audio.get_sample_size(pyaudio.paInt16))
wave_file.setframerate(sample_rate)
wave_file.writeframes(b''.join(frames))
wave_file.close()

KeyboardInterrupt: 

In [19]:

import soundfile as sf
import numpy as np
import torch
from transformers import Wav2Vec2FeatureExtractor, Wav2Vec2Processor, Wav2Vec2Model

feature_extractor = Wav2Vec2FeatureExtractor.from_pretrained("facebook/wav2vec2-base-960h")
processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-base-960h")
model = Wav2Vec2Model.from_pretrained("facebook/wav2vec2-base-960h")


Some weights of the model checkpoint at facebook/wav2vec2-base-960h were not used when initializing Wav2Vec2Model: ['lm_head.bias', 'lm_head.weight']
- This IS expected if you are initializing Wav2Vec2Model from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2Model from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2Model were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [3]:
import soundfile as sf
import numpy as np
import torch
import keras as keras
from collections import Counter

class_to_number = {'Block': 0, 'Fluent': 1, 'Interjection': 2, 'Prolongation': 3, 'SoundRep': 4, 'WordRep': 5}
modell = keras.models.load_model(f"E:\GP\Stuttering Classification\Wav2vec\modelBILSTM1.h5")
# Define the chunk size (in seconds)
chunk_size = 3
audio, sample_rate = sf.read("/GP/Stuttering Classification/")
# Calculate the number of chunks
num_chunks = int(np.ceil(len(audio) / (chunk_size * sample_rate)))

# Initialize the features and predictions arrays
features = []
predictions = []

# Loop over the audio chunks
for i in range(num_chunks):
    # Extract the current chunk of audio
    start_idx = i * chunk_size * sample_rate
    end_idx = min(len(audio), (i + 1) * chunk_size * sample_rate)
    chunk_audio = audio[start_idx:end_idx]

    # Extract features from the current chunk of audio
    input_values = processor(chunk_audio, sampling_rate=sample_rate, return_tensors="pt").input_values
    with torch.no_grad():
        feature = model(input_values).last_hidden_state
    features.append(feature)
    if(feature.size()[1] != 149):
        del features[i]
    # Make a prediction on the current chunk of audio
    chunk_features = np.array(feature)
    chunk_prediction = np.argmax(modell.predict(chunk_features), axis=-1)
    predictions.append(chunk_prediction)

# Combine the predictions into single arrays
predictions = np.concatenate(predictions, axis=0)
print(predictions)
# Save the predictions
#np.save("audio_file_predictions.npy", predictions)


LibsndfileError: Error opening 'E:\\GP\\Stuttering Classification\\Wav2vec\recording2.wav': System error.

In [30]:

count = Counter(predictions)
max_count = max(count.values())
max_predictions = [key for key, value in count.items() if value == max_count][0]
value = [key for key, val in class_to_number.items() if val == max_predictions][0]
print(f"You have a stutter of the type: \033[1m{value}\033[0m")

You have a stutter of the type: WordRep
